# 「枝刈り（Pruning）のチュートリアル」

【原題】Pruning Tutorial

【原著】[Michela Paganini](https://github.com/mickypaganini)

【元URL】https://pytorch.org/tutorials/intermediate/pruning_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年1月26日

【チュトーリアル概要】

最先端のディープラーニング技術は、非常に多くのパラメータを持つモデルとなっています。

このような技術を用いたモデルには、デプロイが困難であるという課題があります。

一方で、生物学的な脳内のニューラルネットワークは、効率的で疎な接続を利用することで知られています。

パラメータ数を削減することによってモデルを圧縮する最適なテクニックを把握しておくことは、精度を損なわずにメモリ、バッテリー、そしてハードウェアの消耗を減らし、デバイス上に軽量なモデルをデプロイするために重要です。

また、プライベートなデバイス（エッジ系）において演算を行う際にプライバシーを確保する上でも重要となります。

枝刈りは、ニューラルアーキテクチャの探索テクニックとして、多くのパラメータで構成されているネットワークと、パラメータが少ないネットワークでの学習ダイナミクスの違いを調査するケースや、疎な当たりのサブネットワークと初期化("[当たりくじ](https://arxiv.org/abs/1803.03635)")の役割を研究するケースなど、研究の最前線で使用されています。

本チュートリアルでは、`torch.nn.utils.prune` を用いてニューラルネットワークを疎にする方法、及びその方法を任意の枝刈りのテクニックの実装に拡張する方法を学びます。

## 必須要件
`"torch>=1.4.0a0+8e8a5e0"`

In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

## モデルの作成

本チュートリアルでは、LeCun 1998らの[LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf)アーキテクチャを使用します。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1つの画像チャネル、6つの出力チャネル、3x3の四角形の畳み込みカーネル
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5の画像次元
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

## モジュールの確認

LeNetモデルの（枝刈りされていない）`conv1`層を確認してみましょう。
`conv1`層は、`weight` と `bias` の2つのパラメータを含んでおり、この時点でバッファは含んでいません。

In [ ]:
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.3273,  0.2712, -0.0953],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.2530,  0.0524]]],


        [[[-0.0895,  0.0482,  0.2638],
          [-0.2001, -0.1045,  0.2797],
          [-0.3160,  0.3137, -0.1420]]],


        [[[-0.0745,  0.2557, -0.0125],
          [ 0.0688,  0.2205,  0.2299],
          [-0.0944,  0.1338, -0.0233]]],


        [[[-0.0590, -0.2994, -0.1391],
          [-0.0028,  0.2432, -0.2896],
          [-0.0729, -0.2522, -0.2231]]],


        [[[ 0.1020, -0.0075, -0.1868],
          [ 0.2437,  0.0663, -0.0489],
          [ 0.2144, -0.1072, -0.3217]]],


        [[[-0.0846,  0.1886,  0.0569],
          [-0.2620, -0.2686, -0.0275],
          [-0.0430,  0.2285, -0.3022]]]], device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
tensor([ 0.1315,  0.1902,  0.1169, -0.0872, -0.0136,  0.2106], device='cuda:0',
       requires_grad=True))]


In [ ]:
print(list(module.named_buffers()))

[]


## モジュールの枝刈り

モジュール（今回の例では、LeNetアーキテクチャの`conv1`層）を枝刈りするには、初めに`torch.nn.utils.prune`（または、`BasePruningMethod`をサブクラス化することで独自に実装したもの）で利用できる選択肢から枝刈りのテクニックを指定します。

そして、モジュールと当該モジュール内で枝刈りするパラメータを設定します。

最後に、選択した枝刈りのテクニックに必要なキーワード引数を用いて、枝刈りを行う際のパラメータを指定します。



今回の例では、`conv1`層の`weight`という名前のパラメータ内の接続をランダムに30％枝刈りします。

モジュールは、枝刈りする関数に最初の引数として渡されます。

その他の引数として、`name` は文字列の識別子を用いてモジュール内のパラメータを識別し、`amount`は、（0. から 1.の間のfloatの場合は）枝刈りする接続のパーセンテージ、または（非負のintegerの場合は）枝刈りする接続の絶対数を示します。

In [ ]:
prune.random_unstructured(module, name="weight", amount=0.3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

パラメータから`weight`を取り除き、`weight_orig`という新たなパラメータに置き換えることで、枝刈りが行われます（例：初期のパラメータの`name`に`"_orig"`を付与されます）。

`weight_orig`は、枝刈りが行われていないバージョンのテンソルを保持しています。

一方で、`bias`は枝刈りされず、そのままの状態を維持します。

In [ ]:
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([ 0.1315,  0.1902,  0.1169, -0.0872, -0.0136,  0.2106], device='cuda:0',
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.3273,  0.2712, -0.0953],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.2530,  0.0524]]],


        [[[-0.0895,  0.0482,  0.2638],
          [-0.2001, -0.1045,  0.2797],
          [-0.3160,  0.3137, -0.1420]]],


        [[[-0.0745,  0.2557, -0.0125],
          [ 0.0688,  0.2205,  0.2299],
          [-0.0944,  0.1338, -0.0233]]],


        [[[-0.0590, -0.2994, -0.1391],
          [-0.0028,  0.2432, -0.2896],
          [-0.0729, -0.2522, -0.2231]]],


        [[[ 0.1020, -0.0075, -0.1868],
          [ 0.2437,  0.0663, -0.0489],
          [ 0.2144, -0.1072, -0.3217]]],


        [[[-0.0846,  0.1886,  0.0569],
          [-0.2620, -0.2686, -0.0275],
          [-0.0430,  0.2285, -0.3022]]]], device='cuda:0', requires_grad=True))]


上で選択された枝刈りのテクニックによって生成された枝刈りのマスクは、`weight_mask`という名前のモジュールバッファとして保存されます（例：初期のパラメータの`name`に`"_mask"`が付与されます）。

In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 0.],
          [0., 1., 0.],
          [1., 1., 0.]]],


        [[[1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 1.],
          [1., 0., 1.],
          [1., 0., 0.]]],


        [[[1., 0., 1.],
          [1., 1., 1.],
          [0., 0., 1.]]],


        [[[0., 1., 0.],
          [1., 1., 0.],
          [1., 1., 0.]]]], device='cuda:0'))]


何も手を加えずにフォワードパスを機能させるには、モジュールに`weight`属性が存在している必要があります。

`torch.nn.utils.prune`で実装されている枝刈りのテクニックは、（マスクを元のパラメータと突き合わせることで）枝刈りされたバージョンの重みを処理し、それらを`weight`属性に格納します。

なお、上記のような枝刈りのテクニックを適用した後は、重みが`module`のパラメータではなく、ただの属性変数になっている点に注意してください。

In [ ]:
print(module.weight)

tensor([[[[ 0.0000,  0.0000, -0.0000],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.0000,  0.0524]]],


        [[[-0.0895,  0.0482,  0.0000],
          [-0.0000, -0.1045,  0.0000],
          [-0.3160,  0.3137, -0.0000]]],


        [[[-0.0745,  0.0000, -0.0000],
          [ 0.0688,  0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.1391],
          [-0.0028,  0.0000, -0.2896],
          [-0.0729, -0.0000, -0.0000]]],


        [[[ 0.1020, -0.0000, -0.1868],
          [ 0.2437,  0.0663, -0.0489],
          [ 0.0000, -0.0000, -0.3217]]],


        [[[-0.0000,  0.1886,  0.0000],
          [-0.2620, -0.2686, -0.0000],
          [-0.0430,  0.2285, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


最後に、各フォワードパスに先立って、PyTorchの`forward_pre_hooks`を使用することで枝刈りを適用します。

具体的には、上で行ったように、`module`が枝刈りされる際、対応するパラメータが枝刈りされる度に `forward_pre_hook` が作成されます。

今回のケースでは、元が`weight`という名前のパラメータのみを枝刈りしたため、一つのフックが存在することになります。

In [ ]:
print(module._forward_pre_hooks)

OrderedDict([(1, <torch.nn.utils.prune.PruningContainer object at 0x7f937097dba8>)])


全パラメータに枝刈りを行うために`bias`も枝刈りしましょう。

`module`のパラメータ、バッファ、フック、そして属性がどのように変わるか確認することができます。

別の枝刈りのテクニックを試すこととし、ここでは`l1_unstructured`という枝刈り関数で実装されている手法を使い、L1ノルムを基準にしてバイアス内の3つの最小の要素を枝刈りします。

In [ ]:
prune.l1_unstructured(module, name="bias", amount=3)

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

これで名前付きパラメータに、`weight_orig`と `bias_orig`の両方が含まれているはずです。

またバッファは、`weight_mask` と `bias_mask` を含んでいます。

そして、枝刈りされたバージョンの2つのテンソルはモジュールの属性として存在し、この時点でモジュールは2つの`forward_pre_hooks`を保有しています。

In [ ]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.3273,  0.2712, -0.0953],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.2530,  0.0524]]],


        [[[-0.0895,  0.0482,  0.2638],
          [-0.2001, -0.1045,  0.2797],
          [-0.3160,  0.3137, -0.1420]]],


        [[[-0.0745,  0.2557, -0.0125],
          [ 0.0688,  0.2205,  0.2299],
          [-0.0944,  0.1338, -0.0233]]],


        [[[-0.0590, -0.2994, -0.1391],
          [-0.0028,  0.2432, -0.2896],
          [-0.0729, -0.2522, -0.2231]]],


        [[[ 0.1020, -0.0075, -0.1868],
          [ 0.2437,  0.0663, -0.0489],
          [ 0.2144, -0.1072, -0.3217]]],


        [[[-0.0846,  0.1886,  0.0569],
          [-0.2620, -0.2686, -0.0275],
          [-0.0430,  0.2285, -0.3022]]]], device='cuda:0', requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.1315,  0.1902,  0.1169, -0.0872, -0.0136,  0.2106], device='cuda:0',
       requires_grad=True))]


In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 0.],
          [0., 1., 0.],
          [1., 1., 0.]]],


        [[[1., 0., 0.],
          [1., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 1.],
          [1., 0., 1.],
          [1., 0., 0.]]],


        [[[1., 0., 1.],
          [1., 1., 1.],
          [0., 0., 1.]]],


        [[[0., 1., 0.],
          [1., 1., 0.],
          [1., 1., 0.]]]], device='cuda:0')), ('bias_mask', tensor([1., 1., 0., 0., 0., 1.], device='cuda:0'))]


In [ ]:
print(module.bias)

tensor([0.1315, 0.1902, 0.0000, -0.0000, -0.0000, 0.2106], device='cuda:0',
       grad_fn=<MulBackward0>)


In [ ]:
print(module._forward_pre_hooks)

OrderedDict([(1, <torch.nn.utils.prune.PruningContainer object at 0x7f937097dba8>), (2, <torch.nn.utils.prune.L1Unstructured object at 0x7f9370978320>)])


## 枝刈りの反復

複数回に渡って、モジュール内の同一のパラメータを枝刈りすることも可能です。

様々な枝刈りの実行により発生する効果は、様々なマスクを順に適用した場合と等しい結果をもたらします。

新たなマスクと古いマスクの組み合わせは、`PruningContainer`の`compute_mask`メソッドによって処理します。

例えば今回は、チャネルのL2ノルムに基づいてテンソルの0番目の軸（0番目の軸は、畳み込み層の出力チャネルに対応しており、`conv1`の場合は6つの要素を有します。）に沿った構造的な枝刈りを行い、`module.weight`をさらに枝刈りしたいとします。

これらの一連の処理は、`ln_structured`関数の引数に `n=2` と `dim=0` を渡すことによって行えます。

In [ ]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# 出力を確認するとわかるように、
# 以前のマスクの実行結果を保持した状態で
# チャネルの50%（6分の3）に対応する接続がゼロ化されています。

print(module.weight)

tensor([[[[ 0.0000,  0.0000, -0.0000],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.0000,  0.0524]]],


        [[[-0.0895,  0.0482,  0.0000],
          [-0.0000, -0.1045,  0.0000],
          [-0.3160,  0.3137, -0.0000]]],


        [[[-0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.1886,  0.0000],
          [-0.2620, -0.2686, -0.0000],
          [-0.0430,  0.2285, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


この時、対応するフックは `torch.nn.utils.prune.PruningContainer`型になり、`weight`パラメータに適用された枝刈りの履歴を保持します。

In [ ]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # 適切なフックを選択
        break

print(list(hook))  # コンテナ内の枝刈りの履歴

[<torch.nn.utils.prune.RandomUnstructured object at 0x7f937097dfd0>, <torch.nn.utils.prune.RandomUnstructured object at 0x7f937097dcf8>, <torch.nn.utils.prune.LnStructured object at 0x7f9370978710>]


## 枝刈りされたモデルのシリアル化

マスクバッファを含む、関連するすべてのテンソルと枝刈りされたテンソルの演算に使用される元のパラメータは、モデルの`state_dict`に格納されているため、必要に応じて簡単にシリアル化や保存を行うことが可能です。

In [ ]:
print(model.state_dict().keys())

odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


## 枝刈りの再パラメータ化の除去

枝刈りを永続的なものにした上で、`weight_orig` と `weight_mask` の再パラメータ化を除去し、`forward_pre_hook`も除去するには、`torch.nn.utils.prune`の`remove`関数を使用します。

なお、これは何も起こらなかったかのように、枝刈りをキャンセルしているわけではない点に注意してください。

キャンセルするのではなく、パラメータ`weight`を、枝刈りされたバージョンのモデルのパラメータに再代入することで、枝刈りを適用し、永続的なものにします。

再パラメータ化の除去前：

In [ ]:
print(list(module.named_parameters()))

[('weight_orig', Parameter containing:
tensor([[[[ 0.3273,  0.2712, -0.0953],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.2530,  0.0524]]],


        [[[-0.0895,  0.0482,  0.2638],
          [-0.2001, -0.1045,  0.2797],
          [-0.3160,  0.3137, -0.1420]]],


        [[[-0.0745,  0.2557, -0.0125],
          [ 0.0688,  0.2205,  0.2299],
          [-0.0944,  0.1338, -0.0233]]],


        [[[-0.0590, -0.2994, -0.1391],
          [-0.0028,  0.2432, -0.2896],
          [-0.0729, -0.2522, -0.2231]]],


        [[[ 0.1020, -0.0075, -0.1868],
          [ 0.2437,  0.0663, -0.0489],
          [ 0.2144, -0.1072, -0.3217]]],


        [[[-0.0846,  0.1886,  0.0569],
          [-0.2620, -0.2686, -0.0275],
          [-0.0430,  0.2285, -0.3022]]]], device='cuda:0', requires_grad=True)), ('bias_orig', Parameter containing:
tensor([ 0.1315,  0.1902,  0.1169, -0.0872, -0.0136,  0.2106], device='cuda:0',
       requires_grad=True))]


In [ ]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0.],
          [1., 1., 1.],
          [1., 0., 1.]]],


        [[[1., 1., 0.],
          [0., 1., 0.],
          [1., 1., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 1., 0.],
          [1., 1., 0.],
          [1., 1., 0.]]]], device='cuda:0')), ('bias_mask', tensor([1., 1., 0., 0., 0., 1.], device='cuda:0'))]


In [ ]:
print(module.weight)

tensor([[[[ 0.0000,  0.0000, -0.0000],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.0000,  0.0524]]],


        [[[-0.0895,  0.0482,  0.0000],
          [-0.0000, -0.1045,  0.0000],
          [-0.3160,  0.3137, -0.0000]]],


        [[[-0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.1886,  0.0000],
          [-0.2620, -0.2686, -0.0000],
          [-0.0430,  0.2285, -0.0000]]]], device='cuda:0',
       grad_fn=<MulBackward0>)


再パラメータ化の除去後：

In [ ]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias_orig', Parameter containing:
tensor([ 0.1315,  0.1902,  0.1169, -0.0872, -0.0136,  0.2106], device='cuda:0',
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000,  0.0000, -0.0000],
          [ 0.0764, -0.2651, -0.3249],
          [ 0.2157, -0.0000,  0.0524]]],


        [[[-0.0895,  0.0482,  0.0000],
          [-0.0000, -0.1045,  0.0000],
          [-0.3160,  0.3137, -0.0000]]],


        [[[-0.0000,  0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.0000,  0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]]],


        [[[ 0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000]]],


        [[[-0.0000,  0.1886,  0.0000],
          [-0.2620, -0.2686, -0.0000],
          [-0.0430,  0.2285, -0.0000]]]], device='cuda:0', requires_grad=True))]


In [ ]:
print(list(module.named_buffers()))

[('bias_mask', tensor([1., 1., 0., 0., 0., 1.], device='cuda:0'))]


## モデル内の複数パラメータの枝刈り 

本チュートリアルを通して確認できるように、理想的な枝刈りのテクニックと対象のパラメータを指定することで、条件にもよりますが、ネットワーク内の複数のテンソルの枝刈りを簡単に行うことができます。

In [ ]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # すべての2次元畳み込み層の接続の20%を枝刈り 
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # すべての線形層の接続の40%を枝刈り 
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # すべてのマスクが存在することを確認

dict_keys(['conv1.weight_mask', 'conv2.weight_mask', 'fc1.weight_mask', 'fc2.weight_mask', 'fc3.weight_mask'])


## グローバルな枝刈り

ここまでの解説では、いわゆる"ローカル"な枝刈りの概念のみを説明しました。
（日本語訳注：モジュールごとに適用するという意味でローカルです）

例えば、各要素の統計量（重みの大きさ，活性化，勾配など）を、そのテンソルの他の要素と排他的に比較することによって，モデルのテンソルを1つずつ枝刈りするような処理です。

しかし、一般的かつ、より強力なテクニックは、（例えば）各層の接続の最小20%を除去するのではなく、モデル全体に渡って最小20%の接続を除去することで、モデルを一度に枝刈りすることです。

グローバルに枝刈りを行った場合は、恐らく層ごとに異なる枝刈りの割合になります。

`torch.nn.utils.prune`の`global_unstructured`関数を使って、グローバルに枝刈りを行う方法を確認しましょう。

In [ ]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

これで、枝刈りされた各パラメータ内に生じたスパース性を確認できます。

各層において20％のスパース性を確保できているわけではありませんが、グローバルなスパース性は（およそ）20%です。

In [ ]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in conv1.weight: 1.85%
Sparsity in conv2.weight: 7.87%
Sparsity in fc1.weight: 21.95%
Sparsity in fc2.weight: 12.66%
Sparsity in fc3.weight: 10.36%
Global sparsity: 20.00%


## オリジナルの枝刈りの関数を用いた`torch.nn.utils.prune`の拡張

独自の枝刈りの関数を実装するには、他のすべての枝刈りのメソッドが行っている方法と同様、`BasePruningMethod`の基底クラスをサブクラス化することで`nn.utils.prune`を拡張できます。

基底クラスには、`__call__`、`apply_mask`、`apply`、`prune`、そして`remove` といったメソッドが実装されています。

いくつかの特殊なケースを除いては、新しい枝刈りのテクニックのためにこれらのメソッドを再実装する必要は生じません。

しかし、`__init__`（コンストラクター）と`compute_mask`（枝刈りのテクニックのロジックに応じた、所与のテンソルに対するマスクを処理する方策）は実装する必要があります。



さらに、どのタイプのテクニックの実装にするかを指定する必要があります（`global`、`structured`、そして`unstructured`の選択肢がサポートされています。）。

これは、枝刈りが繰り返し適用された場合に、どのようにマスクを組み合わせるか判断するために必要な準備になります。

言い換えれば、既に枝刈りされたパラメータを枝刈りする場合、その時点で使用されている枝刈りのテクニックはパラメータの枝刈りされていない部分に作用することが期待されています。

`PRUNING_TYPE`を指定することで、（枝刈りマスクの反復適用を扱う）`PruningContainer`が枝刈りするパラメータの断面を適切に識別できるようにします。

例えば、テンソル内で一つおきに枝刈りを行うテクニックを実装したいとしましょう（または、テンソルが既に枝刈りされていた場合は、そのテンソルの残りの枝刈りされていない部分）。

この場合、`PRUNING_TYPE='unstructured'` とします。

なぜならば枝刈りの対象が、ユニット/チャネル全体 に対して(`'structured'`)ではなく、異なるパラメータに渡る(`'global'`)なわけでもなく、層内の個別の接続に対して行われるためです。


In [ ]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """
    テンソル内で一つおきに枝刈りを行う
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0 
        return mask

では、`nn.Module`内のパラメータにこれを適用するために、メソッドをインスタンス化し、そのメソッドを適用する簡単な関数を準備しましょう。

In [ ]:
def foobar_unstructured(module, name):
    """
    テンソル内のエントリを1つおきに除去することで、
    `module`内の`name`というパラメータに対応するテンソルを枝刈りする。
    以下の要領でin-placeにモジュールを変更する（そして変更されたモジュールを返す）。
    1) 枝刈りのメソッドによって、`name`パラメータに適用されたバイナリのマスクに対応する
    `name+'_mask'` という名前付きバッファを加える。
    `name`パラメータは枝刈りされたバージョンに置換される一方で、
    元の（枝刈りされていない）パラメータは、`name+'_orig'`という名前の新しいパラメータに格納される。

    Args:
        module (nn.Module): 枝刈りの対象となるテンソルを含むモジュール
        name (string): 枝刈りが作用する対象となる`module`内のパラメータ名

    Returns:
        module (nn.Module): 変更（例：枝刈り）されたバージョンの入力モジュール
    
    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

試してみましょう！

In [ ]:
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])


以上。